In [39]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import os

import pickle

from tmu.models.autoencoder.autoencoder import TMAutoEncoder
from src.lib.care import calculate_care_score

In [40]:
bits = 5

In [41]:
# Create folder figures_huber if it does not exist
os.makedirs("figures_huber", exist_ok=True)

In [42]:
def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take the first 3000 rows
    #X = X[:-6000]

    return X


def load_thresh_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take the first 5000 rows
    X = X[:10000]

    return X


def load_test_labels(farm, event_id):
    # Load dataframe from file
    df = pd.read_csv(f"./data_test/y_{farm}_{event_id}.csv")

    labels = df['label'].values
    status_ids = df['status_type_id'].values
    train_test = df['train_test'].values

    # Take the first 3000 rows
    #labels = labels[:-6000]
    #status_ids = status_ids[:-6000]
    #train_test = train_test[:-6000]

    return np.array(labels).astype(np.uint32), np.array(status_ids).astype(np.uint32), train_test


def load_test_label(farm, event_id):
    event_info = pd.read_csv(f"../../../data/care_to_compare/Wind Farm {farm}/event_info.csv", delimiter=';')

    metadata = event_info[event_info['event_id'] == event_id]

    event_label = metadata["event_label"].values[0]

    return False if event_label == "anomaly" else True

In [43]:
def calculate_accuracy(labels, predictions):
    # Calculate the accuracy
    accuracy = np.sum(labels == predictions) / len(labels)

    return accuracy


def calculate_threshold(X, pred):
    losses = [huber_loss(X[i], pred[i]) for i in range(len(X))]

    # Set the threshold as the lowest 1% of the losses
    threshold = np.percentile(losses, 80)

    return threshold


In [44]:
def load_model(filename) -> TMAutoEncoder:
    with open(filename, "rb") as f:
        model = pickle.load(f)

    return model

In [45]:
def hamming_loss(pred, X_test):
    """
    Computes the Hamming loss between predicted and ground truth binary arrays.

    Parameters:
    - pred (numpy array): Binary predictions of shape (n_samples, n_bits).
    - X_test (numpy array): Ground truth binary values of shape (n_samples, n_bits).

    Returns:
    - float: Hamming loss (fraction of incorrect bits).
    """
    assert pred.shape == X_test.shape, "Shapes of pred and X_test must match"

    # Compute the number of differing bits
    incorrect_bits = np.sum(pred != X_test)

    # Total number of bits
    total_bits = np.prod(X_test.shape)

    # Hamming loss is the fraction of incorrect bits
    return incorrect_bits / total_bits


def binary_to_decimal(arr, bit_length):
    # Split the array into chunks of bit_length
    numbers = [int("".join(map(str, arr[i:i + bit_length])), 2) for i in range(0, len(arr), bit_length)]
    return numbers


def huber_loss(pred, X_test, delta=1.0):
    # Reconstruct the original values (5 bits)
    p = binary_to_decimal(pred, bits)
    x = binary_to_decimal(X_test, bits)

    # Compute the Huber loss
    loss = np.where(np.abs(np.array(p) - np.array(x)) < delta, 0.5 * ((np.array(p) - np.array(x)) ** 2),
                    delta * (np.abs(np.array(p) - np.array(x)) - 0.5 * delta))

    return np.mean(loss)

In [46]:
def reconstruction_accuracy(X, pred):
    correct = np.sum(X == pred)
    accuracy = correct / len(X)
    return accuracy


def plot_mse(X, y, pred, name, threshold):
    # Compute MSE for each row
    mse_per_row = [huber_loss(X[i], pred[i]) for i in range(len(X))]

    # Plot SNS plot of all MSE values
    plt.figure(figsize=(8, 4))
    sns.histplot(mse_per_row, bins=50, kde=True, color='b')

    # Add a threshold line
    plt.axvline(threshold, color='r', linestyle='--')

    # Save the plot
    plt.savefig(f"./figures_huber/plot_reconstruction_acc_{name}.png")

    plt.close()


def plot_predictions(X, y, z, pred, p, name, threshold):
    x = np.arange(0, len(X))  # Time or index
    r = [huber_loss(X[i], pred[i]) for i in range(len(X))]

    anomalies = np.array([1 if (p[i] == 1 and z[i] == 0) else 0 for i in range(len(X))])

    y_mapped = np.where(y == 0, -0.2, -0.1)
    a_mapped = np.where(anomalies == 0, -0.4, -0.3)
    p_mapped = np.where(p == 0, -0.6, -0.5)
    z_mapped = np.where(np.logical_or(z == 0, z == 2), -0.8, -0.7)

    # Create a figure with two subplots (1 row, 2 columns)
    fig, axes = plt.subplots(2, 1, figsize=(12, 10))

    # First chart
    # If z is 0 or 2, then the color is orange, if y[i] is 1, then the color is red, otherwise blue
    colors = ['red' if y[i] == 1 else 'blue' if z[i] == 0 or z[i] == 2 else 'orange' for i in range(len(z))]

    axes[0].scatter(x, r, label="Reconstruction Loss", c=colors, alpha=0.7)
    axes[0].axhline(threshold, color='red', linestyle='--', label="Threshold")

    axes[0].set_title("Reconstruction Loss")
    axes[0].set_xlabel("Time")
    axes[0].set_ylabel("Reconstruction Loss")
    axes[0].legend()

    # Second chart
    axes[1].plot(x, y_mapped, label="Actual Anomalies (y)", color='red', linestyle='-', linewidth=2)
    axes[1].plot(x, a_mapped, label="Detected Anomalies (a)", color='purple', linestyle='-', linewidth=2)
    axes[1].plot(x, p_mapped, label="Predicted Anomalies (p)", color='green', linestyle='-', linewidth=2)
    axes[1].plot(x, z_mapped, label="Status Type Id (z)", color='orange', linestyle='-', linewidth=2)

    axes[1].axhline(y=-0.1, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.2, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.3, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.4, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.5, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.6, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.7, color='black', linestyle='dotted', linewidth=1, alpha=0.5)
    axes[1].axhline(y=-0.8, color='black', linestyle='dotted', linewidth=1, alpha=0.5)

    axes[1].set_ylim(-0.9, 0)

    axes[1].set_title("Anomalies")
    axes[1].legend()

    # Adjust layout
    plt.tight_layout()
    plt.grid(True)

    # plt.show()

    # Save the plot
    plt.savefig(f"./figures_huber/plot_detections_{name}.png")

    plt.close()


def get_predictions(X, y, z, name, tm, threshold):
    pred = tm.predict(X)

    # For each row in pred, if the MSE is greater than the threshold, then it is an anomaly
    losses = [huber_loss(X[i], pred[i]) for i in range(len(X))]

    X_predictions = np.array([1 if losses[i] > threshold else 0 for i in range(len(X))])

    # Accuracy
    accuracy = calculate_accuracy(y, X_predictions)

    plot_mse(X, y, pred, name, threshold)
    plot_predictions(X, y, z, pred, X_predictions, name, threshold)

    return X_predictions, accuracy

In [47]:
test_datasets = [34, 7, 53, 27, 19, 77, 83, 52, 21, 2, 23, 87, 74, 86, 82]

tm_autoencoder = load_model("models/latest_24.pkl")

X_thresh = load_thresh_dataset("B", 83)
X_thresh_pred = tm_autoencoder.predict(X_thresh)

threshold = calculate_threshold(X_thresh, X_thresh_pred)

print(f"Threshold: {threshold}")


def run_prediction(farm, dataset):
    X = load_test_dataset(farm, dataset)
    labels, status_ids, train_test = load_test_labels(farm, dataset)

    is_normal = load_test_label(farm, dataset)

    predictions, accuracy = get_predictions(X, labels, status_ids, f"{farm}_{dataset}", tm_autoencoder, threshold)

    return X, labels, status_ids, train_test, is_normal, predictions, accuracy


Threshold: 18.702380952380953


In [48]:
# Create a dataframe with status_type_id;label;prediction
elements = []

for set in test_datasets:
    res = run_prediction("B", set)

    result_df = pd.DataFrame({
        'status_type_id': res[2],
        'label': res[1],
        'prediction': res[5],
        'train_test': res[3],
    })

    print(f"Done with {set}. Accuracy: {res[6]}")

    elements.append({'dataset': set, 'normal': res[4], 'data': result_df, 'accuracy': res[6]})

Done with 34. Accuracy: 0.7863267507027563
Done with 7. Accuracy: 0.8543390749987042
Done with 53. Accuracy: 0.7472954987543937
Done with 27. Accuracy: 0.6725076693275084
Done with 19. Accuracy: 0.745597355747872
Done with 77. Accuracy: 0.5616531494342188
Done with 83. Accuracy: 0.7456955844784061
Done with 52. Accuracy: 0.8341324841225324
Done with 21. Accuracy: 0.8271473424534649
Done with 2. Accuracy: 0.912848978399401
Done with 23. Accuracy: 0.7874626427824939
Done with 87. Accuracy: 0.7678910188079279
Done with 74. Accuracy: 0.7652560205751695
Done with 86. Accuracy: 0.5645254316715331
Done with 82. Accuracy: 0.7972264686612798


In [49]:
# Safe the results to results.pkl
with open("results_full_99.pkl", "wb") as f:
    pickle.dump(elements, f)

In [52]:
import pickle
from src.lib.care import calculate_care_score

# Load elements from results_full_99.pkl
with open("results_full_99.pkl", "rb") as f:
    elements = pickle.load(f)

score = calculate_care_score(elements)
print(score)

0.6700125829419117


In [51]:
# Trained on only two 5 bit datasets: 0.6608247996790653
# Trained on all 5 bit datasets: 0.6661408285906466